__Must first find all Dublin EV Charge Points__ via `extract-esb-ev-chargepoint-locations.ipynb`

# Get specific chargepoint data

The ESB yields specific charge-point information (such as Power in kW) only when the user clicks on the charge point in the Web-App.  To simulate this we can make a POST request for each charge point id.  

__Note__: To find out what information the ESB's web map sends to Google Maps (i.e. a POST request) to get charge point coordinates to display;
- `firefox` inspect network tool to observe the POST request
- Right-click copy `curl` command
- Paste to https://curl.trillworks.com/ to convert this request to `python` code which generates the `headers` and `data` inputs below

... retry request if fails https://stackoverflow.com/questions/15431044/can-i-set-max-retries-for-requests-request/15431343#15431343

In [112]:
from glob import glob
import json
from random import randint
from time import sleep

import requests
from tqdm import tqdm

In [ ]:
dublin_ev_chargepoints = gpd.read_file("../data/dublin_ev_chargepoint_locations")

In [32]:
dublin_ev_chargepoint_ids = dublin_ev_chargepoints["id"].tolist()

In [156]:
def save_chargepoint_id_post_request(id, filepath):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:83.0) Gecko/20100101 Firefox/83.0',
        'Accept': 'application/json, text/javascript, */*; q=0.01',
        'Accept-Language': 'en-US,en;q=0.5',
        'Content-Type': 'application/json',
        'Origin': 'https://esb.ie',
        'DNT': '1',
        'Connection': 'keep-alive',
        'Referer': 'https://esb.ie/ecars/DrivezMap.html?53.2980656,-6.1752800,14z,Irish%20Rail%20Booterstown%20DART%20Station,%20Rock%20Road,%20Blackrock,%20Dublin,198si',
        'Sec-GPC': '1',
    }
    data = '{"filterByIds":[' + str(id) + ']}'
    try:
        response = requests.post(
            'https://myaccount.esbecars.com/stationFacade/findStationsByIds',
            headers=headers,
            data=data
        )
        with open(filepath, "w") as json_file:
            json.dump(response.json(), json_file)
    except:
        print(f"Request for {id} was unsuccessful")

In [130]:
for id in tqdm(dublin_ev_chargepoint_ids):
    save_chargepoint_id_post_request(id, f"dublin_ev_chargepoint_data/{id}.json")
    sleep(randint(2, 5)) # sleep a random number between each request so don't get shut down    

100%|██████████| 1/1 [00:05<00:00,  5.55s/it]


In [152]:
unsuccessful_responses = []
successful_responses = []
filepaths = Path("dublin_ev_chargepoint_data")

for filepath in filepaths.glob("*.json"):
    with open(filepath) as json_file:
        recorded_response = json.load(json_file)
    
    try:
        sucessful_response = recorded_response["data"][1][0]
        successful_responses.append(sucessful_response)
    except:
        unsuccessful_responses.append(filepath.stem)

In [145]:
station_data = pd.DataFrame(successful_responses)

In [146]:
station_data

,@class,id,deleted,dirty,caption,latitude,longitude,isManaged,stationStatusId,addressCountryName,...,addressAddress1,addressCity,addressRegion,stationModelInstructionsVideoUrl,chargingSpeedId,stationSockets,stationReservations,inMaintenance,canRegisterForNotifyWhenAvailable,markForReplacement
0,com.driivz.stationserver.bl.dto.imp.StationDtoImp,189,False,False,C6BTQ,51.709117,-8.520562,True,AVAILABLE,Ireland,...,"Fire Brigade Car Park, New Road",Kinsale,Cork,obzL8txRivY,SEMI_FAST,"[{'id': 378, 'deleted': False, 'dirty': False,...",[],False,False,False
1,com.driivz.stationserver.bl.dto.imp.StationDtoImp,221,False,False,C6F5J,52.139237,-8.654588,True,AVAILABLE,Ireland,...,"Irish Rail Mallow Station, Limerick Road",Mallow,Cork,obzL8txRivY,SEMI_FAST,"[{'id': 442, 'deleted': False, 'dirty': False,...",[],False,False,False
2,com.driivz.stationserver.bl.dto.imp.StationDtoImp,334,False,False,C2PDW-A,52.266040,-7.116893,True,AVAILABLE,Ireland,...,"Irish Rail Plunkett Station, Dock Road",Abbeylands,Waterford,exGy3o6TySU,SEMI_FAST,"[{'id': 626, 'deleted': False, 'dirty': False,...",[],False,False,False
3,com.driivz.stationserver.bl.dto.imp.StationDtoImp,521,False,False,RC11,54.995194,-7.306255,True,AVAILABLE,United Kingdom,...,"Maxol Service Station, Glendermot Road, Waterside",Londonderry,NaN,q39p1KVTO7M,FAST,"[{'id': 866, 'deleted': False, 'dirty': False,...",[],False,False,False
4,com.driivz.stationserver.bl.dto.imp.StationDtoImp,1285,False,False,C8F8V,53.415451,-6.834811,True,AVAILABLE,Ireland,...,"Irish Rail Train Station Car Park, Off Main St...",Enfield,Meath,liMu_I2_iJw,SEMI_FAST,"[{'id': 2262, 'deleted': False, 'dirty': False...",[],False,False,False
5,com.driivz.stationserver.bl.dto.imp.StationDtoImp,80,False,False,C6RN7,52.658200,-8.629400,True,AVAILABLE,Ireland,...,Pery Square,Limerick City,Limerick,obzL8txRivY,SEMI_FAST,"[{'id': 160, 'deleted': False, 'dirty': False,...",[],False,False,False
6,com.driivz.stationserver.bl.dto.imp.StationDtoImp,237,False,False,C9FY9,53.622633,-9.221264,True,AVAILABLE,Ireland,...,"Supervalue Car Park, Church Lane",Ballinrobe,Mayo,obzL8txRivY,SEMI_FAST,"[{'id': 474, 'deleted': False, 'dirty': False,...",[],False,False,False
7,com.driivz.stationserver.bl.dto.imp.StationDtoImp,19,False,False,CBTQR,52.241464,-7.169840,True,AVAILABLE,Ireland,...,"Top/Londis Service Station, Holy Cross/Butlers...",Waterford City,Waterford,q39p1KVTO7M,FAST,"[{'id': 40, 'deleted': False, 'dirty': False, ...",[],False,False,False
8,com.driivz.stationserver.bl.dto.imp.StationDtoImp,379,False,False,C79FH,53.358200,-6.267200,True,AVAILABLE,Ireland,...,Nelson Street,Dublin,Dublin 7,WCIo_0tXKTE,SEMI_FAST,"[{'id': 672, 'deleted': False, 'dirty': False,...",[],False,False,False
9,com.driivz.stationserver.bl.dto.imp.StationDtoImp,93,False,False,C7KH4,53.329333,-8.220936,True,AVAILABLE,Ireland,...,Saint Michael's Square,Ballinasloe,Galway,obzL8txRivY,SEMI_FAST,"[{'id': 186, 'deleted': False, 'dirty': False,...",[],False,False,False
